In [1]:
import pandas as pd
import numpy as np
from time import time
import re
import os
import nltk
import gensim.models.word2vec as w2v

nltk.download("punkt")
nltk.download("stopwords")

from nltk.corpus import stopwords

from nltk import word_tokenize
from nltk.stem import PorterStemmer

import multiprocessing

import seaborn as sns
import gensim.models.word2vec as w2v
import matplotlib.pyplot as plt
import sklearn.manifold

from  sklearn.cluster import KMeans
import glob
import gensim 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
file = os.listdir("../data_scripts/_data/insta_captions/")
file

['insta_1502353915.txt', 'insta_1502360336.txt']

In [3]:
all_data = []
for i in file:
    with open("../data_scripts/_data/insta_captions/{}".format(i)) as f:
        temp_data = f.read().split('\n')
    all_data.extend(temp_data)

In [4]:
def clean_sentence(raw):
    noise_words = []
    stop_words = list(set(stopwords.words('english')))
    URL_REGEX = r"""(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?<>""'']))"""
    raw = re.sub(URL_REGEX, '', raw, flags=re.MULTILINE)
    for n in noise_words:
        clean = raw.replace(n, " ").replace("#", "")
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    stop_words = list(set(stopwords.words('english')))
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

In [5]:
df = pd.DataFrame(all_data, columns=['captions'])
df.replace('', np.nan, inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.tail()
df = pd.DataFrame(df['captions'].apply(clean_sentence))
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.count()

captions    54343
dtype: int64

In [6]:
df.head()

,captions
0,Tonight
1,Social Vibes Present URBAN SOUND Thursday Augu...
2,
3,event eventparty dj djparty night nightlife eo...
4,Life Metro


In [7]:
def sentence_to_wordlist(text, stem=True):
    """ Tokenize text and stem words removing punctuation """
    tokens = word_tokenize(text.lower())
    return tokens

In [8]:
sentences = []
for raw_sentence in df['captions']:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))

In [9]:
sentences[909]

['newarrival',
 'top',
 'bottom',
 'dresses',
 'accessories',
 'shoes',
 'bag',
 'malaysia',
 'fashion',
 'online',
 'onlineshop',
 'chicstylishboutique',
 'instagood',
 'instalike',
 'instadaily',
 'instafashion',
 'instafollow',
 'followus',
 'follow',
 'following',
 'freepostage',
 'korea',
 'style']

In [11]:
token_count = sum([len(sentence) for sentence in sentences])
print("The corpus contains {0:,} tokens".format(token_count))

The corpus contains 728,930 tokens


In [17]:
# Dimensionality of the resulting word vectors.
# more dimensions mean more training them, but more generalized
num_features = 300
# Minimum word count threshold.
min_word_count = 3
# Number of threads to run in parallel.
num_workers = multiprocessing.cpu_count()
# Context window length.
context_size = 7
# Downsample setting for frequent words.
#rate 0 and 1e-5 
#how often to use
downsampling = 1e-3
# Seed for the RNG, to make the results reproducible.
seed = 1

In [18]:
t0 = time()
bio_vec = w2v.Word2Vec(
    sentences,
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling,
    iter = 500
)

print("Train Time:", time()-t0)

Train Time: 1114.1881289482117


In [19]:
if not os.path.exists("_data/trained"):
    os.makedirs("_data/trained")

bio_vec.save(os.path.join("_data/trained", "tagsVec_{}.w2v".format(int(time()))))

In [63]:
bio_vec.most_similar_cosmul("pydata")

KeyError: "word 'pydata' not in vocabulary"